In [1]:
import sqlite3
import psycopg2
import pandas as pd
from DataBase_functions import *
from DataBase_functions_testing import *

#Google sheed API
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

<p style="font-size: 30px; text-align: center; color: skyblue;">Creating data-frame</p>

In [10]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c" #Registro actividad todos: 12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #Dataframe con los datos de todos los meses
    df = pd.DataFrame(columns=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]) 
    meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
    for sheet in sheets:
        sheet_name = sheet.title 
        if sheet_name in meses:
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
            df = pd.concat([df, df_aux], ignore_index=True)
        else: 
            print(sheet_name, "is not a month")
            pass
    #df = df.drop_duplicates(subset='Código', keep='first')
    print(df.shape)

    #Dataframe with 2024 sheet data
    sheet_2024 = spreadsheet.worksheet("2024")
    df_2024 = pd.DataFrame(sheet_2024.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
    #df_2024 = df_2024.drop_duplicates(subset='Código', keep='first')
    print(df_2024.shape)

    df = pd.concat([df, df_2024], ignore_index=True)
    df = df.drop_duplicates(subset="Código", keep="first")
    print(df.shape)

    #Dataframe validation
    df.rename(columns={"Validada": "validada", "Responsable": "responsable", "Gestor DDBB": "DDBB", "Geografía": "geography", "Petición": "petition_code", "Feature": "feature", "SDATOOL": "sdatool", "Código": "DQDP_code", "Gestor BBDD": "DDBB", "Horas": "duration_time", "Comentarios": "description", "Fecha de alta": "fecha_in", "Fecha de fin": "fecha_out"}, inplace=True)

    df["fecha_in"] = df["fecha_in"].astype(str)
    df["Fecha incurrida"] = df["Fecha incurrida"].astype(str)
    df["fecha_out"] = df["fecha_out"].astype(str)
    df["UUAA"] = df["UUAA"].astype(str)
    df["DQDP_code"] = df["DQDP_code"].astype(str)
    df["sdatool"] = df["sdatool"].astype(str)
    df["feature"] = df["feature"].astype(str)
    df["geography"] = df["geography"].astype(str)
    df["DDBB"] = df["DDBB"].astype(str)
    df["Ámbito"] = df["Ámbito"].astype(str)
    df["responsable"] = df["responsable"].astype(str)
    df["validada"] = df["validada"].astype(str)
    df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
    df["description"] = df["description"].astype(str)

    df["version_date"] = "Nan"
    df["version"] = "Nan"
    df["petition_arq"] = "Nan"
    df["dev_master"] = "Nan"
    df["estado"] = "Nan"

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['2024 totales - Peticiones', '2024 totales - Peticiones - Pro', '2024', 'Gráficos Semanales - Procesos', '2024 - Procesos', 'Abril', 'Mayo', 'Julio', 'Junio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Responsables']
2024 totales - Peticiones is not a month
2024 totales - Peticiones - Pro is not a month
2024 is not a month
Gráficos Semanales - Procesos is not a month
2024 - Procesos is not a month
Responsables is not a month
(4371, 18)
(1974, 18)
(1739, 18)


In [11]:
for row in df.iterrows(): 
    for key, value in dict(row[1]).items():
        if value == "Nan" or value == None or value == "" or value == "None": 
            if key == "petition_code" or key == "DQDP_code" or key == "sdatool" or key == "feature" or key == "UUAA" or key == "version" or key == "petition_arq" or key == "description":
                df.at[row[0], key] = "None"
            else: 
                df.at[row[0], key] = None
print(df.shape, df.columns)
print(df.isna().sum())

(1739, 23) Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'duration_time', 'description', 'Estado',
       'Días', '', 'version_date', 'version', 'petition_arq', 'dev_master',
       'estado'],
      dtype='object')
fecha_in            263
Fecha incurrida      55
fecha_out           391
UUAA                  0
DQDP_code             0
sdatool               0
feature               0
petition_code         0
geography           268
DDBB                170
Ámbito              170
responsable         631
validada            854
duration_time        72
description           0
Estado             1150
Días               1296
                    458
version_date       1739
version               0
petition_arq          0
dev_master         1739
estado             1739
dtype: int64


In [12]:
for row in df.iterrows():
    insert_data_testing(dict(row[1]))

OperationalError: table Power_Design has no column named version_date

<p style="font-size: 40px; color: grey;">Testing</p>

In [5]:
diccionario = {"Nombre": "Matias", "Apellido": "Blaquier", "Edad": 27}
for key, value in diccionario.items(): 
    print(value)

###########
str_test = " "
print(len(str_test))
print(len(str_test.strip()))

Matias
Blaquier
27
1
0
